# boardcnn.py EXPLAINED

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image
import tensorflow as tf

In [ ]:
from clusterization import clusterize
from chessutils import find_coeffs

In [ ]:
def intersect(i1, i2):
    return (i2[0] <= i1[1]) and (i1[0] <= i2[1])
def closeintervals(i1, i2):
    return intersect(i1, i2) and (abs(i1[2]-i2[2]) == 1)

In [ ]:
def centerofmass(c):
    xm = 0
    ym = 0
    nm = 0
    for i in c:
        ns = i[1]-i[0]+1
        xm += 0.5*(i[1]+i[0])*ns
        ym += i[2]*ns
        nm += ns
    return xm/nm, ym/nm

In [ ]:
colors = np.array([' ', 'w', 'b'])
figures = np.array([' ','p','b','n','r','q','k'])

In [ ]:
NUMCELL = 8
CELL = 60
IMGSIZE = NUMCELL*CELL

In [ ]:
#Loading models
model = tf.keras.models.load_model('models/unet_board_v4.h5')
classifier = tf.keras.models.load_model('models/class_figure_col_v2.h5', compile=False)

In [ ]:
#Folder with images
PATH_TO_TST = 'out'

In [ ]:
#Images in folder
fl = [os.path.join(PATH_TO_TST, f) for f in os.listdir(PATH_TO_TST) if f.split('.')[-1] == 'png']
fl

In [ ]:
filename = fl[0]

In [ ]:
#Loading image
img = cv2.imread(filename)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img, (IMGSIZE,IMGSIZE))
feed = np.expand_dims(img/255, axis=(0,-1))
feed.shape

In [ ]:
#Detecting characteristic points
pred = np.squeeze(model.predict(feed))
points = np.argmax(pred, axis=-1)

In [ ]:
#Visualizing
fig, axx = plt.subplots(nrows=1, ncols=2, figsize=(10,5), sharey=True)
axx[0].imshow(img, cmap='gray')
axx[1].imshow(points, cmap='gray')
fig.tight_layout()
plt.show()

In [ ]:
#Clusterizing points
invpoints = 1-points
xsumms = np.sum(invpoints, axis=-1)
res = []
for y, xsumm in enumerate(xsumms):
    if xsumm == 0:
        continue
    ##TODO extend with zeros??
    s = invpoints[y]
    diffs = s[1:]-s[:-1]
    starts = np.where(diffs==1)[0]
    ends = np.where(diffs==-1)[0]
    for i1, i2 in zip(starts,ends):
        res.append([i1+1,i2,y])
cl = clusterize(res, closeintervals)

In [ ]:
#Visualizing
clcol = np.zeros((IMGSIZE,IMGSIZE), dtype=np.int32)
for i, c in enumerate(cl):
    for k in c:
        clcol[k[2],k[0]:k[1]+1] = i+1

fig, axx = plt.subplots(nrows=1, ncols=2, figsize=(10,5), sharey=True, sharex=True)
axx[0].imshow(points, cmap='gray')
axx[1].imshow(clcol, vmin=0, vmax=len(cl))
fig.tight_layout()
plt.show()

In [ ]:
#Centers of clusters
xy = np.array(list(map(centerofmass, cl)))
#Mutual center
xyc = np.mean(xy, axis=0)

In [ ]:
#(Manhattan) distances between centers and xyc
mhdists = np.array([np.sum(np.abs(p-xyc)) for p in xy])
#Most distant clusters
corners = xy[np.argsort(mhdists)[-4:]]

In [ ]:
#half of diagonal
half = 0.5*np.max(mhdists)
#new corners
newc = xyc + half*np.array([[-1,-1],[1,-1],[-1,1],[1,1]])

In [ ]:
#matching between initial and new corners
ixmatch = [np.argmin(np.sum(np.abs(p-corners), axis=1)) for p in newc]
cornersmatch = corners[ixmatch]

In [ ]:
#widening board
brdsize = half / 6 * NUMCELL
boardcorners = xyc + brdsize*np.array([[-1,-1],[1,-1],[-1,1],[1,1]])

In [ ]:
#Transformation coefficients
coeffs = find_coeffs(newc, cornersmatch)

In [ ]:
#Transforming/cropping
img = Image.fromarray(img)
img = img.transform(img.size, Image.PERSPECTIVE, coeffs, Image.BICUBIC, fill='white')
img = img.crop((xyc[0]-brdsize, xyc[1]-brdsize, xyc[0]+brdsize, xyc[1]+brdsize))

In [ ]:
#resizing
img = img.resize((IMGSIZE,IMGSIZE))

In [ ]:
#Separating by fields
cropped = np.asarray(img)
stacked = np.zeros((NUMCELL*NUMCELL,CELL,CELL))
for i in range(NUMCELL*NUMCELL):
    xp = i % NUMCELL
    yp = i // NUMCELL
    stacked[i] = cropped[yp*CELL:(yp+1)*CELL,xp*CELL:(xp+1)*CELL]/255
stacked = np.expand_dims(stacked,axis=-1)

In [ ]:
#predicting figures and colors
preds = classifier.predict(stacked)

In [ ]:
#Predicted figures
figures[np.argmax(preds[:,0:7],axis=-1)].reshape((8,8))

In [ ]:
#Predicted colors
colors[np.argmax(preds[:,7:10],axis=-1)].reshape((8,8))

In [ ]:
img.resize((120,120))